In [1]:
import pandas as pd

In [2]:
train_df=pd.read_json('C:/Users/SHAM/OneDrive/Desktop/project/trained_data_extractive_summaries.json', lines=True)
test_df=pd.read_json('C:/Users/SHAM/OneDrive/Desktop/project/tested_data_with_clean_extractive_summaries.json', lines=True)



In [3]:

valid_df=pd.read_json('C:/Users/SHAM/OneDrive/Desktop/project/valid_data_with_clean_extractive_summaries.json', lines=True)


In [4]:
len(train_df), len(test_df), len(valid_df)

(9117, 1953, 1302)

In [5]:
train_df=train_df.head(100)
test_df=test_df.head(100)
valid_df=valid_df.head(100)

In [6]:
train_df['extractive_summary'][0]

'టామ్ ప్రైస్ నిబంధనల జాతీయ భద్రత వ్యవహారాలను చూసే ప్రభుత్వ అధికారులు విధి నిర్వహణకు సంబంధించిన ప్రయాణాలకు సాధారణ వాణిజ్య విమానాలనే ఉపయోగించాలి ప్రైవేటు విమాన ప్రయాణాలకు పెట్టిన ఖర్చును చెల్లిస్తానన్నారు అమెరికా వెలుపల ప్రయాణాలకు ఉపయోగించిన సైనిక విమాన ఖర్చు వ్యయంలో ఉంది అధ్యక్షుడు డొనాల్డ్ ట్రంప్ బృందంలో ప్రైవేటు విమానాలను ఉపయోగించిన మరో ముగ్గురి విషయం ప్రస్తుతం పరిశీలనలో ఉంది ముగ్గురిలో అంతర్గత వ్యవహారాల మంత్రి ర్యాన్ జింకే ఆర్థిక శాఖ మంత్రి స్టీవెన్ నుచిన్ పర్యావరణ పరిరక్షణ ఏజెన్సీ చీఫ్ స్కాట్ ప్రూయిట్ ర్యాన్ జింకే ప్రైస్ ప్రైవేటు విమానాల ప్రయాణ ఖర్చుపై అధ్యక్షుడు ట్రంప్ ఇంతకుముందు అసంతృప్తి వ్యక్తంచేశారు ప్రైస్ రాజీనామాను అధ్యక్షుడు ఆమోదించినట్లు వైట్ హౌస్ ప్రకటనలో తెలిపింది ఆరోగ్య శాఖ ఉప సహాయ మంత్రిగా డాన్ జే రైట్ను తాత్కాలిక మంత్రిగా నియమించినట్లు చెప్పింది ట్రంప్ ఆశిస్తున్నట్లుగా ఒబామా కేర్ రద్దును ప్రైస్ సాధించి పెట్టి రాజీనామా వచ్చేది కాదేమోనని అభిప్రాయపడ్డారు'

In [7]:
len(train_df['extractive_summary'][0].split())

106

In [8]:
train_df['summary'][0]

'విధి నిర్వహణకు ఖరీదైన ప్రైవేటు విమానాలను ఉపయోగించినందుకు విమర్శలను ఎదుర్కొన్న అమెరికా ఆరోగ్యశాఖ మంత్రి టామ్ ప్రైస్ తన పదవికి రాజీనామా చేశారు.'

In [9]:
from collections import Counter
def tokenize(text):
    return text.strip().split()

def build_vocab(texts, min_freq=1, specials=["<pad>", "<unk>", "<sos>", "<eos>"]):
    counter = Counter()
    for text in texts:
        counter.update(tokenize(text))
    tokens = [tok for tok, freq in counter.items() if freq >= min_freq]
    itos = specials + sorted(tokens)
    stoi = {tok: idx for idx, tok in enumerate(itos)}
    return {"stoi": stoi, "itos": itos}


In [10]:
src_vocab_data={**build_vocab(train_df['extractive_summary']),**build_vocab(valid_df["extractive_summary"]), **build_vocab(test_df["extractive_summary"])}

In [ ]:

tgt_vocab_data ={**build_vocab(train_df['summary']))
src_vocab, src_itos = src_vocab_data["stoi"], src_vocab_data["itos"]
tgt_vocab, tgt_itos = tgt_vocab_data["stoi"], tgt_vocab_data["itos"]


In [13]:
def numericalize(text, vocab):
    tokens = tokenize(text)
    return [vocab["<sos>"]] + [vocab.get(tok, vocab["<unk>"]) for tok in tokens] + [vocab["<eos>"]]


In [14]:

def decode(indices, itos):
    return " ".join([itos[idx] for idx in indices if 0 <= idx < len(itos) and idx != src_vocab["<pad>"]])

#    return " ".join([itos[idx] for idx in indices if idx != src_vocab["<pad>"]])

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
class SummarizationDataset(Dataset):
    def __init__(self, df):
        self.data = df
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        src = numericalize(self.data.iloc[idx]['extractive_summary'], src_vocab)
        tgt = numericalize(self.data.iloc[idx]['summary'], tgt_vocab)
        return torch.tensor(src), torch.tensor(tgt)


In [16]:
def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=src_vocab["<pad>"])
    tgt_batch = pad_sequence(tgt_batch, padding_value=tgt_vocab["<pad>"])
    return src_batch, tgt_batch


In [17]:

train_ds = SummarizationDataset(train_df)


In [18]:
train_ds.data

,id,url,title,summary,text,extractive_summary
0,international-41451124,https://www.bbc.com/telugu/international-41451124,విమాన ప్రయాణాల వివాదం: మూల్యం చెల్లించుకున్నప్...,విధి నిర్వహణకు ఖరీదైన ప్రైవేటు విమానాలను ఉపయోగ...,[విమాన ప్రయాణాల వివాదం మూల్యం చెల్లించుకున్నప్...,టామ్ ప్రైస్ నిబంధనల జాతీయ భద్రత వ్యవహారాలను చూ...
1,india-57341250,https://www.bbc.com/telugu/india-57341250,ప్రపంచ సైకిల్ దినోత్సవం: తొలి సైకిల్ ఎప్పుడు ప...,ఏటా ఎన్ని సైకిళ్లు తయారవుతున్నాయో తెలుసా? అసలు...,[ప్రపంచ సైకిల్ దినోత్సవం తొలి సైకిల్ పరుగులు త...,ప్రపంచ సైకిల్ దినోత్సవం సందర్భంగా ఆసక్తికరమైన ...
2,india-53160266,https://www.bbc.com/telugu/india-53160266,కరోనావైరస్ వ్యాక్సిన్: భారత ‘వ్యాక్సిన్ కింగ్’...,"కరోనా దెబ్బకు వ్యాపారాలన్నీ కుదేలై, దేశ ఆర్థిక...",[కరోనావైరస్ వ్యాక్సిన్ భారత వ్యాక్సిన్ కింగ్ ప...,కరోనావైరస్ వ్యాక్సిన్ భారత వ్యాక్సిన్ కింగ్ పూ...
3,india-44465377,https://www.bbc.com/telugu/india-44465377,'ముందు నీళ్లు అందించండి.. ఆ తర్వాత మరుగుదొడ్లు...,డుంబ్రిగుడ మండలంలోని కొర్రాయి కొత్తవలస గ్రామాన...,"[నీళ్లు అందించండి.. ఆ మరుగుదొడ్లు వాడుతాం, మరు...",నీళ్లు అందించండి.. ఆ మరుగుదొడ్లు వాడుతాం మరుగు...
4,india-54982946,https://www.bbc.com/telugu/india-54982946,కోవిడ్-19: మౌత్‌వాష్‌తో 30 సెకన్లలోనే కరోనావైర...,కరోనావైరస్‌ను కేవలం 30 సెకన్లలోనే మౌత్‌వాష్‌లు...,[కోవిడ్19 మౌత్వాష్తో 30 సెకన్లలోనే కరోనావైరస్ ...,కోవిడ్19 మౌత్వాష్తో 30 సెకన్లలోనే కరోనావైరస్ హ...
...,...,...,...,...,...,...
95,india-50970539,https://www.bbc.com/telugu/india-50970539,CAA - NRC: ఆందోళనకారుల ఆస్తులను ప్రభుత్వం జప్త...,పౌరసత్వ సవరణ చట్టానికి వ్యతిరేకంగా ఇటీవల దేశవ్...,[CAA NRC ఆందోళనకారుల ఆస్తులను ప్రభుత్వం జప్తు ...,ప్రభుత్వ ఆస్తులకు నష్టం కలిగించిన వారిపై ప్రతీ...
96,india-52007915,https://www.bbc.com/telugu/india-52007915,కరోనావైరస్ గ్రామాలకు పాకితే భారత్ పరిస్థితి ఏంటి?,భారత్‌లోని చాలా ప్రాంతాలు లాక్‌డౌన్‌లోకి వెళ్ల...,[కరోనావైరస్ గ్రామాలకు పాకితే భారత్ పరిస్థితి ఏ...,2020 మార్చి 3 వరకూ భారత్ వ్యాప్తంగా ఐదు కరోనాక...
97,india-54340142,https://www.bbc.com/telugu/india-54340142,భారత రత్న జాబితాలో దక్షిణాది వారికి తగిన ప్రామ...,భారత ప్రభుత్వం వివిధ రంగాలలో విశిష్ట సేవలందించ...,[భారత రత్న జాబితాలో దక్షిణాది వారికి ప్రాముఖ్య...,భారత రత్న జాబితాలో దక్షిణాది వారికి ప్రాముఖ్యం...
98,international-49354317,https://www.bbc.com/telugu/international-49354317,భారత్‌తో యుద్ధానికి మేం సిద్ధం.. ఆర్ఎస్ఎస్ హిట...,భారత్‌తో యుద్ధానికి తమ దేశం పూర్తి సింసిద్ధంగా...,[భారత్తో యుద్ధానికి మేం సిద్ధం.. ఆర్ఎస్ఎస్ హిట...,భారత్తో యుద్ధానికి మేం సిద్ధం.. ఆర్ఎస్ఎస్ హిట్...


In [17]:
src_vocab_data1 = build_vocab(valid_df["extractive_summary"])
tgt_vocab_data1 = build_vocab(valid_df["summary"])
src_vocab1, src_itos1= src_vocab_data1["stoi"], src_vocab_data1["itos"]
tgt_vocab1, tgt_itos1 = tgt_vocab_data1["stoi"], tgt_vocab_data1["itos"]


In [19]:

val_ds = SummarizationDataset(valid_df)


In [20]:

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)


In [21]:

val_dl = DataLoader(val_ds, batch_size=32, collate_fn=collate_fn)


In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=src_vocab["<pad>"])
        self.rnn = nn.GRU(emb_dim, hid_dim)
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden


In [24]:

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Parameter(torch.rand(hid_dim))
    def forward(self, hidden, encoder_outputs):
        src_len = encoder_outputs.shape[0]
        hidden = hidden[-1].unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs.permute(1, 0, 2)), dim=2)))
        energy = energy.permute(0, 2, 1)
        v = self.v.repeat(encoder_outputs.size(1), 1).unsqueeze(1)
        attention = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention, dim=1)


In [25]:

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=tgt_vocab["<pad>"])
        self.rnn = nn.GRU(hid_dim + emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim * 2 + emb_dim, output_dim)
        self.attention = attention

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        a = self.attention(hidden, encoder_outputs).unsqueeze(1)
        weighted = torch.bmm(a, encoder_outputs.permute(1, 0, 2))
        rnn_input = torch.cat((embedded, weighted.permute(1, 0, 2)), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        output = output.squeeze(0)
        embedded = embedded.squeeze(0)
        weighted = weighted.squeeze(1)
        prediction = self.fc(torch.cat((output, weighted, embedded), dim=1))
        return prediction, hidden


In [26]:

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        tgt_len, batch_size = tgt.shape
        output_dim = self.decoder.output_dim
        outputs = torch.zeros(tgt_len, batch_size, output_dim).to(src.device)
        encoder_outputs, hidden = self.encoder(src)
        input = tgt[0,:]
        for t in range(1, tgt_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = tgt[t] if teacher_force else top1
        return outputs


In [27]:

# ------------------ Training ------------------
INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(tgt_vocab)
HID_DIM = 256
EMB_DIM = 256


In [28]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [29]:

attn = Attention(HID_DIM)


In [30]:

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, attn)
model = Seq2Seq(enc, dec).to(device)


In [31]:
model.eval

<bound method Module.eval of Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(12019, 256, padding_idx=0)
    (rnn): GRU(256, 256)
  )
  (decoder): Decoder(
    (embedding): Embedding(1633, 256, padding_idx=0)
    (rnn): GRU(512, 256)
    (fc): Linear(in_features=768, out_features=1633, bias=True)
    (attention): Attention(
      (attn): Linear(in_features=512, out_features=256, bias=True)
    )
  )
)>

In [33]:
from torch import nn, optim
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])

In [34]:
from torch.nn.utils.rnn import pad_sequence


In [35]:

def train(model, iterator):
    model.train()
    total_loss = 0
    for src, tgt in iterator:
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt)
        output = output[1:].reshape(-1, output.shape[-1])
        tgt = tgt[1:].reshape(-1)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(iterator)

for epoch in range(10):
    loss = train(model, train_dl)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


Epoch 1, Loss: 5.9998
Epoch 2, Loss: 3.2000
Epoch 3, Loss: 2.7993
Epoch 4, Loss: 2.5425
Epoch 5, Loss: 2.4279
Epoch 6, Loss: 2.5018
Epoch 7, Loss: 2.5349
Epoch 8, Loss: 2.3289
Epoch 9, Loss: 2.4071
Epoch 10, Loss: 2.3840


In [36]:
def evaluate(model, iterator):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in iterator:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt, teacher_forcing_ratio=0)
            output = output[1:].reshape(-1, output.shape[-1])
            tgt = tgt[1:].reshape(-1)
            loss = criterion(output, tgt)
            total_loss += loss.item()
    return total_loss / len(iterator)

# Example usage:
for epoch in range(10):
    val_loss = evaluate(model, val_dl)
    print(f"Validation Loss: {val_loss:.4f}")

Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559
Validation Loss: 3.0559


In [37]:

# ------------------ Save ------------------
torch.save(model.state_dict(), "telugu_seq2seq2.pt")


In [43]:

# ------------------ Inference ------------------
def summarize(text, max_len=64):
    #model.eval()
    model.load_state_dict(torch.load("telugu_seq2seq2.pt"))
    model.to(device)
    model.eval()
    tokens = numericalize(text, src_vocab)
    src_tensor = torch.tensor(tokens).unsqueeze(1).to(device)
    with torch.no_grad():
        enc_out, hidden = model.encoder(src_tensor)
    input_tok = torch.tensor([tgt_vocab["<sos>"]]).to(device)
    result = []
    for _ in range(max_len):
        output, hidden = model.decoder(input_tok, hidden, enc_out)
        top1 = output.argmax(1).item()
        if top1 == tgt_vocab["<eos>"]:
            break
        result.append(top1)
        input_tok = torch.tensor([top1]).to(device)
    return decode(result, tgt_itos)


In [39]:
valid_df

,id,url,title,summary,text,words,extractive_summary,cleaned_summary
0,india-43961357,https://www.bbc.com/telugu/india-43961357,మే డే: కార్మికులు ఏ దేశాల్లో ఎక్కువ సేపు పనిచే...,కార్మికులు ధనిక దేశాల్లో ఎక్కువ సమయం పనిచేస్తు...,[మే డే కార్మికులు ఏ దేశాల్లో ఎక్కువ సేపు పనిచే...,"[మే, డే, కార్మికులు, ఏ, దేశాల్లో, ఎక్కువ, సేపు...",వారానికి గరిష్ఠంగా 48 లేదా అంతకంటే తక్కువ పనిగ...,వారానికి గరిష్ఠంగా 48 లేదా అంతకంటే పనిగంటలు ఉం...
1,international-49974060,https://www.bbc.com/telugu/international-49974060,"నోబెల్ ప్రైజ్: విశ్వ రహస్యాల శోధనకు, కొత్త గ్ర...",విశ్వానికి సంబంధించి సరికొత్త అంశాలను కనుగొన్న...,[నోబెల్ ప్రైజ్ విశ్వ రహస్యాల శోధనకు కొత్త గ్రహ...,"[నోబెల్, ప్రైజ్, విశ్వ, రహస్యాల, శోధనకు, కొత్త...",నోబెల్ ప్రైజ్ విశ్వ రహస్యాల శోధనకు కొత్త గ్రహా...,నోబెల్ ప్రైజ్ విశ్వ రహస్యాల శోధనకు కొత్త గ్రహా...
2,india-47940194,https://www.bbc.com/telugu/india-47940194,'ది మోస్ట్ బ్యూటిఫుల్ ఫేస్ ఆన్ ది ఇండియన్ స్క్...,తెలుగు సినీరంగాన్ని కొన్నేళ్ల పాటు ఏలిన కథానాయ...,[ది మోస్ట్ బ్యూటిఫుల్ ఫేస్ ఆన్ ది ఇండియన్ స్క్...,"[ది, మోస్ట్, బ్యూటిఫుల్, ఫేస్, ఆన్, ది, ఇండియన...",ది మోస్ట్ బ్యూటిఫుల్ ఫేస్ ఆన్ ది ఇండియన్ స్క్ర...,ది మోస్ట్ బ్యూటిఫుల్ ఫేస్ ఆన్ ది ఇండియన్ స్క్ర...
3,india-45788302,https://www.bbc.com/telugu/india-45788302,గండికోట జలాశయం: ''మా ఇళ్లలోకి నీరొస్తోంది.. పర...,శ్రీశైలం జలాశయం నుంచి కడప జిల్లాలోని గండికోట జ...,[గండికోట జలాశయం మా ఇళ్లలోకి నీరొస్తోంది.. పరిహ...,"[గండికోట, జలాశయం, మా, ఇళ్లలోకి, నీరొస్తోంది, ....",గండికోట జలాశయం మా ఇళ్లలోకి నీరొస్తోంది.. పరిహా...,గండికోట జలాశయం మా ఇళ్లలోకి నీరొస్తోంది.. పరిహా...
4,india-48162157,https://www.bbc.com/telugu/india-48162157,అరవింద్ కేజ్రీవాల్‌ను చెంపదెబ్బ కొట్టిన సురేశ్...,దిల్లీ ముఖ్యమంత్రి అరవింద్ కేజ్రీవాల్‌ ఎన్నికల...,[అరవింద్ కేజ్రీవాల్ను చెంపదెబ్బ కొట్టిన సురేశ్...,"[అరవింద్, కేజ్రీవాల్ను, చెంపదెబ్బ, కొట్టిన, సు...",పోస్ట్ of Twitter ముగిసింది 1 న్యూ దిల్లీ లోక్...,పోస్ట్ of Twitter ముగిసింది 1 న్యూ దిల్లీ లోక్...
...,...,...,...,...,...,...,...,...
95,international-44493024,https://www.bbc.com/telugu/international-44493024,దివ్య సూర్యదేవర: ఒకప్పుడు డబ్బుల్లేక ఇబ్బందులు...,"బీకాం, ఎంకాం చదివితే ఏమవ్వొచ్చు? మన కష్టానికి ...",[దివ్య సూర్యదేవర ఒకప్పుడు డబ్బుల్లేక ఇబ్బందులు...,"[దివ్య, సూర్యదేవర, ఒకప్పుడు, డబ్బుల్లేక, ఇబ్బం...",దివ్య సూర్యదేవర ఒకప్పుడు డబ్బుల్లేక ఇబ్బందులు ...,దివ్య సూర్యదేవర ఒకప్పుడు డబ్బుల్లేక ఇబ్బందులు ...
96,international-43469447,https://www.bbc.com/telugu/international-43469447,శత్రువులపై రక్తసిక్త పోరుకైనా చైనా రెడీ: షీ జి...,తమ దేశానికి చెందిన ఒక అంగుళం భూభాగాన్ని కూడా త...,[శత్రువులపై రక్తసిక్త పోరుకైనా చైనా రెడీ షీ జి...,"[శత్రువులపై, రక్తసిక్త, పోరుకైనా, చైనా, రెడీ, ...",పార్లమెంట్లో ముగింపు ప్రసంగం చేస్తూ చైనాను విభ...,పార్లమెంట్లో ముగింపు ప్రసంగం చేస్తూ చైనాను విభ...
97,international-52698488,https://www.bbc.com/telugu/international-52698488,కరోనావైరస్ ఫుట్‌బాల్: జర్మనీలో ప్రీమియ‌ర్ లీగ్...,జ‌ర్మ‌నీ ఫుట్‌బాల్ ప్రీమియ‌ర్ లీగ్ బుండెస్‌లీగ...,[కరోనావైరస్ ఫుట్బాల్ జర్మనీలో ప్రీమియర్ లీగ్ ఎ...,"[కరోనావైరస్, ఫుట్బాల్, జర్మనీలో, ప్రీమియర్, లీ...",గోల్ కొట్టిన ప్రతిసారీ క్రీడాకారులు హత్తుకొని ...,గోల్ కొట్టిన ప్రతిసారీ క్రీడాకారులు హత్తుకొని ...
98,international-54313643,https://www.bbc.com/telugu/international-54313643,అర్మేనియా - అజర్బైజాన్‌ల మధ్య ‘సరిహద్దు యుద్ధం...,వివాదాస్పద నాగోర్నో-కరబఖ్ సరిహద్దు వద్ద అర్మేన...,[అర్మేనియా అజర్బైజాన్ల మధ్య సరిహద్దు యుద్ధం ...,"[అర్మేనియా, అజర్బైజాన్ల, మధ్య, సరిహద్దు, యుద్ధ...",దేశంలో సైనిక శాసనం మార్షలా లావ్ అమలులోకి తెచ్చ...,దేశంలో సైనిక శాసనం మార్షలా లావ్ అమలులోకి తెచ్చ...


In [44]:
x=test_df['extractive_summary'][5]

In [45]:
x

'దీంతో ఇద్దరు బిడ్డల పోషణ ఆమెకు కష్టంగా మారింది మహిళా సంఘంలో చేరడంతో ఆమె పరిస్థితి మారింది ఇప్పుడామె తన ఇద్దరు బిడ్డలను ఉన్నత చదువులు చదివిస్తున్నారు అంతేకాదు ఉత్తరప్రదేశ్లోని పేద మహిళలకు శిక్షణ ఇస్తూ చేతులా సంపాదిస్తున్నారు జిల్లా మహిళలు ఉత్తరప్రదేశ్లోని మహిళలకు శిక్షణ ఇస్తూ కొత్త చరిత్ర సృష్టిస్తున్నారు ఆర్థికంగా ప్రగతి సాధిస్తున్నారు పాలమూరు నుంచి ఉత్తరప్రదేశ్కు ఒప్పందంలో భాగంగా అక్కడి మహిళా సంఘాల సభ్యులకు శిక్షణ ఇవ్వాల్సి శిక్షణ బాధ్యతలు పాలమూరు మహా సమాఖ్య మహిళా సంఘానికి దక్కాయి ఉత్తరప్రదేశ్లో మహిళలకు శిక్షణ ఇవ్వాలంటే హిందీ తప్పనిసరిగా రావాలి జిల్లా అధికారులు దీన్నో సమస్యగానే భావించలేదు సంఘంలోని కొంతమంది సభ్యులను ఎంపిక చేసి వారికి హిందీలో శిక్షణ ఇచ్చి పంపాలని నిర్ణయించారు 6 లక్షల మంది సభ్యులు దీంతో ఉత్తరప్రదేశ్లో శిక్షణ ఇచ్చే సభ్యులను ఎంపిక చేసేందుకు అధికారులు కొన్ని అర్హతలు నిర్ణయించారు అర్హత సాధించిన వారికి ఇంటర్వ్యూలు నిర్వహించారు ఇలా పరీక్షలు నిర్వహించి వంద మందిని ఎంపిక చేశారు వీరికి నేషనల్ రూరల్ లైవ్లీహుడ్ మిషన్ ద్వారా శిక్షణ ఇప్పించారు హిందీలో మాట్లాడటం రాయడం చదవడంపై ఎక్కువ ద

In [54]:
type(x)

str

In [56]:
test_df['extractive_summary'][0]

'బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఉండబోతోంది ఏడున్నర కోట్ల మంది అమెరికన్లు బైడెన్కు ఓట్లు వేశారు'

In [57]:
test_df

,id,url,title,summary,text,extractive_summary
0,54991770,https://www.bbc.com/telugu/54991770,బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఎలా ఉండబో...,అమెరికా నూతన అధ్యక్షునిగా ఎన్నికైన జో బైడెన్ మ...,[బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఉండబోతోం...,బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఉండబోతోంద...
1,international-55128897,https://www.bbc.com/telugu/international-55128897,కావన్: ఈ పాకిస్తాన్ ఏనుగు భారత్ మీదుగా కంబోడియ...,పాకిస్తాన్‌లోని ఓ జూలో కొన్ని సంవత్సరాల పాటు ఒ...,[కావన్ పాకిస్తాన్ ఏనుగు భారత్ మీదుగా కంబోడియాక...,కావన్ పాకిస్తాన్ ఏనుగు భారత్ మీదుగా కంబోడియాకు...
2,international-50811418,https://www.bbc.com/telugu/international-50811418,ఉల్లి మన ఆహారంలో ఎలా భాగమైంది? దాని చరిత్ర ఏంటి?,"కొంత కాలంగా రోజూ వార్తల్లో కనిపిస్తున్నది, రైత...","[ఉల్లి మన ఆహారంలో భాగమైంది దాని చరిత్ర ఏంటి, ప...",ప్రపంచంలో అనేక దేశాలలో ఉల్లిని పండిస్తున్నారు ...
3,india-56788820,https://www.bbc.com/telugu/india-56788820,తమిళనాడు ఒక చిన్న గ్రామంలో నివసిస్తున్న సిలికా...,"""నేను ఓ రిమోట్ సీఈఓను"" అంటూ గాలికి అటూఇటూ ఊగుత...",[తమిళనాడు చిన్న గ్రామంలో నివసిస్తున్న సిలికాన్...,శ్రీధర్ వెంబు శ్రీధర్ తన సోదరుడితో కలిసి 1996ల...
4,india-55734987,https://www.bbc.com/telugu/india-55734987,చైనా అరుణాచల్ ప్రదేశ్‌లోకి చొచ్చుకువచ్చి ఓ గ్ర...,"చైనా వాస్తవాధీన రేఖను దాటి, భారత్‌లోని అరుణాచల...",[చైనా అరుణాచల్ ప్రదేశ్లోకి చొచ్చుకువచ్చి ఓ గ్ర...,చైనా అరుణాచల్ ప్రదేశ్లోకి చొచ్చుకువచ్చి ఓ గ్రా...
...,...,...,...,...,...,...
95,india-44198053,https://www.bbc.com/telugu/india-44198053,కేరళను వణికిస్తున్న నిపా వైరస్,నిపా వైరస్ కేరళలో తీవ్ర కలకలం సృష్టిస్తోంది.,"[కేరళను వణికిస్తున్న నిపా వైరస్, గబ్బిలాల ద్వా...",కేరళను వణికిస్తున్న నిపా వైరస్ గబ్బిలాల ద్వారా...
96,india-56210280,https://www.bbc.com/telugu/india-56210280,పశ్చిమ బెంగాల్ ఎన్నికలు ఎనిమిది దశల్లో.. మార్చ...,పశ్చిమబెంగాల్ అసెంబ్లీ ఎన్నికల తేదీలను ఎలక్షన్...,[పశ్చిమ బెంగాల్ ఎన్నికలు ఎనిమిది దశల్లో.. మార్...,అయిదో దశ పోలింగ్ ఏప్రిల్ 17న తమిళనాడు తమిళనాడు...
97,india-43268854,https://www.bbc.com/telugu/india-43268854,త్రిపురలో కమల వికాసం.. సుదీర్ఘ కమ్యూనిస్టు పాల...,త్రిపురలో పాతికేళ్ల సీపీఎం పరిపాలన ముగిసింది. ...,[త్రిపురలో కమల వికాసం.. సుదీర్ఘ కమ్యూనిస్టు పా...,మేఘాలయలో కాంగ్రెస్కు మెజార్టీ స్థానాలు లభించాయ...
98,international-52166143,https://www.bbc.com/telugu/international-52166143,కరోనావైరస్: తీవ్ర సంక్షోభం దిశగా పాకిస్తాన్.. ...,"యుద్ధం, సంక్షోభం లాంటి పరిస్థితుల్లో జనం అన్నీ...",[కరోనావైరస్ తీవ్ర సంక్షోభం దిశగా పాకిస్తాన్.. ...,కరోనావైరస్ తీవ్ర సంక్షోభం దిశగా పాకిస్తాన్.. క...


In [48]:
valid_df['summary'][0]

"కార్మికులు ధనిక దేశాల్లో ఎక్కువ సమయం పనిచేస్తున్నారా, పేద దేశాల్లో ఎక్కువ సమయం పనిచేస్తున్నారా? అసలు ప్రపంచవ్యాప్తంగా పనిగంటలు ఎలా ఉన్నాయి? సెలవులు ఎలా ఉన్నాయి? 'మే డే' సందర్భంగా ప్రత్యేక కథనం..."

In [39]:
valid_df['extractive_summary'][0]

'వారానికి గరిష్ఠంగా 48 లేదా అంతకంటే తక్కువ పనిగంటలు ఉండాలని ఐఎల్వో సిఫార్సులు చెబుతున్నాయి దక్షిణ కొరియా చట్టసభ నేషనల్ అసెంబ్లీ దేశంలోని ఉద్యోగులకు కార్మికులకు ఊరట కలించేలా పనిగంటలను తగ్గిస్తూ మార్చిలో ఒక చట్టాన్ని తీసుకొచ్చింది వారానికి గరిష్ఠంగా 68గా ఉన్న పనిగంటలను 52 గంటలకు కుదించింది ప్రజల జీవన ప్రమాణాలు పెంచేందుకు మరిన్ని ఉద్యోగాలు సృష్టించేందుకు ఉత్పాదకతను పెంచేందుకు ఈ చట్టం అవసరమని ప్రభుత్వం భావిస్తోంది పనిగంటల తగ్గింపుతో దేశంలో జననాల రేటు కూడా పెరుగుతుందని ప్రభుత్వం ఆశిస్తోంది గత కొన్ని దశాబ్దాల్లో దక్షిణ కొరియాలో జననాల రేటు బాగా తగ్గింది ఒక్కో కార్మికుడు సగటున ఏడాదికి 2069 గంటలు పనిచేస్తున్నట్లు ఓఈసీడీ నివేదిక తెలిపింది ఓఈసీడీ 38 దేశాలపై ఈ విశ్లేషణ జరిపింది వ్యతిరేకించిన కంపెనీలు కంపెనీల నుంచి వ్యక్తమైన వ్యతిరేకతను పక్కనబెట్టి దక్షిణ కొరియా ప్రభుత్వం ఈ చట్టాన్ని తీసుకొచ్చింది ఇది జులై నుంచి అమలవుతుంది దీనిని తొలి దశలో భారీ కంపెనీలు అమలు చేయాల్సి ఉంటుంది తర్వాత చిన్నస్థాయి కంపెనీలు కూడా దీని ప్రకారం పనిగంటలను తగ్గించాల్సి ఉంటుంది కంపెనీల వ్యతిరేకతను పక్కనబెట్టి దక్షిణ కొరియా ప్

In [51]:
valid_df['text'][0]

['మే డే కార్మికులు ఏ దేశాల్లో ఎక్కువ సేపు పనిచేస్తున్నారు',
 'వారానికి గరిష్ఠంగా 48 లేదా అంతకంటే తక్కువ పనిగంటలు ఉండాలని ఐఎల్వో సిఫార్సులు చెబుతున్నాయి',
 'దక్షిణ కొరియా చట్టసభ నేషనల్ అసెంబ్లీ దేశంలోని ఉద్యోగులకు కార్మికులకు ఊరట కలించేలా పనిగంటలను తగ్గిస్తూ మార్చిలో ఒక చట్టాన్ని తీసుకొచ్చింది',
 'వారానికి గరిష్ఠంగా 68గా ఉన్న పనిగంటలను 52 గంటలకు కుదించింది',
 'ప్రజల జీవన ప్రమాణాలు పెంచేందుకు మరిన్ని ఉద్యోగాలు సృష్టించేందుకు ఉత్పాదకతను పెంచేందుకు ఈ చట్టం అవసరమని ప్రభుత్వం భావిస్తోంది',
 'పనిగంటల తగ్గింపుతో దేశంలో జననాల రేటు కూడా పెరుగుతుందని ప్రభుత్వం ఆశిస్తోంది',
 'గత కొన్ని దశాబ్దాల్లో దక్షిణ కొరియాలో జననాల రేటు బాగా తగ్గింది',
 'ఆర్థిక సహకార అభివృద్ధి సంస్థఆర్గనైజేషన్ ఫర్ ఎకనామిక్ కోఆపరేషన్ అండ్ డెవలప్మెంట్ఓఈసీడీ 2016లో విడుదల చేసిన ఒక నివేదిక ప్రకారం అత్యధిక పనిగంటలున్న అభివృద్ధి చెందిన దేశం దక్షిణ కొరియానే',
 'ఒక్కో కార్మికుడు సగటున ఏడాదికి 2069 గంటలు పనిచేస్తున్నట్లు ఓఈసీడీ నివేదిక తెలిపింది',
 'ఓఈసీడీ 38 దేశాలపై ఈ విశ్లేషణ జరిపింది',
 'వ్యతిరేకించిన కంపెనీలు కంపెనీల నుంచి వ్యక్తమై

In [38]:
# Apply to test set
train_df["generated_summary"] = train_df["extractive_summary"].apply(summarize)
#test_df["generated_summary"][0] = test_df["extractive_summary"][0].split().apply(summarize)


C:\Users\SHAM\AppData\Local\Temp\ipykernel_66952\1638642917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["generated_summary"] = train_df["extractive_summary"].apply(summarize)


In [40]:
train_df['generated_summary'][0]

'విధి నిర్వహణకు ఖరీదైన ప్రైవేటు విమానాలను ఉపయోగించినందుకు విమర్శలను ఎదుర్కొన్న ఎదుర్కొన్న అమెరికా ఆరోగ్యశాఖ మంత్రి టామ్ ప్రైస్'

In [41]:
train_df['summary'][0]

'విధి నిర్వహణకు ఖరీదైన ప్రైవేటు విమానాలను ఉపయోగించినందుకు విమర్శలను ఎదుర్కొన్న అమెరికా ఆరోగ్యశాఖ మంత్రి టామ్ ప్రైస్ తన పదవికి రాజీనామా చేశారు.'

In [52]:
test_df['generated_summary'] = test_df['extractive_summary'].apply(summarize)
test_df['generated_summary'][0]

'ప్రపంచంలోని అత్యంత ధనికులకు, పేదవారికి ఆర్థిక అంతరాలు గణనీయంగా పెరిగాయని ఒక ఏకే-47 రైఫిల్\u200c అప్పగిస్తే,'

In [53]:
test_df['extractive_summary'][0]

'బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఉండబోతోంది ఏడున్నర కోట్ల మంది అమెరికన్లు బైడెన్కు ఓట్లు వేశారు'

In [54]:
test_df['text'][0]

['బైడెన్ నాయకత్వంలో అమెరికా భవిష్యత్తు ఉండబోతోంది',
 'ఏడున్నర కోట్ల మంది అమెరికన్లు బైడెన్కు ఓట్లు వేశారు',
 'మరి భిన్న ధ్రువాలుగా విడిపోయిన ప్రజలను బైడెన్ ఏకం చేయగలరా అందులోనూ కోవిడ్ కేసులు పెరిగిపోతున్న నేపథ్యంలో ప్రజల ఐక్యత అసలు సాధ్యమేనా చేయండి']